In [3]:
# %%
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import xarray as xr
import plotly.graph_objects as go
import dash_ag_grid as dag

# %%
# Initialize the app
app = Dash(__name__)

In [4]:
# %%
# Incorporate data
ds_location = r"C:\Users\Kris\Documents\amazonforcast\data\prakrut\output\LIS_HIST_Forecast_June_02_to_05_mean.nc"
ds = xr.open_dataset(ds_location)

# %%
# define key elements from the dataset
time = ds['time']
longitude = ds['east_west'].values
latitude = ds['north_south'].values
# SoilTemp_profiles = ds['SoilTemp_profiles']
# SoilMoist_profiles = ds['SoilMoist_profiles']

# list of variables in the data set
list_of_variables = ['Rainf_tavg', 'Qair_f_tavg',
                     'Qs_tavg','Evap_tavg', 
                     'SoilMoist_inst', 'SoilTemp_inst'] 

In [6]:
ds

<xarray.Dataset>
Dimensions:           (time: 4, north_south: 540, east_west: 660,
                       SoilMoist_profiles: 4, SoilTemp_profiles: 4)
Coordinates:
  * time              (time) datetime64[ns] 2024-06-02 2024-06-03 ... 2024-06-05
  * north_south       (north_south) float64 -21.0 -20.95 -20.9 ... 5.9 5.95 6.0
  * east_west         (east_west) float64 -82.0 -81.95 -81.9 ... -49.05 -49.0
Dimensions without coordinates: SoilMoist_profiles, SoilTemp_profiles
Data variables: (12/32)
    lat               (time, north_south) float32 ...
    lon               (time, east_west) float32 ...
    Swnet_tavg        (time, north_south, east_west) float32 ...
    Lwnet_tavg        (time, north_south, east_west) float32 ...
    Qle_tavg          (time, north_south, east_west) float32 ...
    Qh_tavg           (time, north_south, east_west) float32 ...
    ...                ...
    Qair_f_tavg       (time, north_south, east_west) float32 ...
    Psurf_f_tavg      (time, north_south, east_west) float32 ...
    SWdown_f_tavg     (time, north_south, east_west) float32 ...
    LWdown_f_tavg     (time, north_south, east_west) float32 ...
    LAI_inst          (time, north_south, east_west) float32 ...
    Greenness_inst    (time, north_south, east_west) float32 ...

In [7]:
selected_variable = ds['SoilMoist_inst']
selected_variable

<xarray.DataArray 'SoilMoist_inst' (time: 4, SoilMoist_profiles: 4,
                                    north_south: 540, east_west: 660)>
[5702400 values with dtype=float32]
Coordinates:
  * time         (time) datetime64[ns] 2024-06-02 2024-06-03 ... 2024-06-05
  * north_south  (north_south) float64 -21.0 -20.95 -20.9 ... 5.9 5.95 6.0
  * east_west    (east_west) float64 -82.0 -81.95 -81.9 ... -49.1 -49.05 -49.0
Dimensions without coordinates: SoilMoist_profiles

In [8]:
soilmoist_at_1 = selected_variable.isel(SoilMoist_profiles = 1)

In [10]:
var_sizes = ds['SoilMoist_inst'].sizes
var_sizes = var_sizes('SoilMoist_profiles')


TypeError: 'Frozen' object is not callable

In [3]:
# %%
# App layout
app.layout = html.Div([
    html.H1(children='Mapping - Forecast Data'),
    html.Hr(),
    dcc.Slider(0, 3, step=4, value=0, marks={0: '2024-06-02', 1:'2024-06-03', 2:'2024-06-04', 3:'2024-06-05'}, id='time_index'),
    html.H2(children='Select your variable below'),
    dcc.RadioItems(options=list_of_variables, value='Rainf_tavg', id='var-selector'),
    html.H2(children='Depth'),
    dcc.Dropdown(id='profile-selector', className='',
        options=[{'label': '0-10cm', 'value': '0'}, {'label': '10-40cm', 'value': '1' }, 
                 {'label': '40-100cm', 'value': '2'}, {'label': '100-200cm', 'value': '3'}],
        value='0',
        #placeholder='0'
    ),
    html.Hr(),
    dcc.Graph(id='graph1')
])

In [6]:
# %%
# Add controls to build the interaction
@callback(
    Output(component_id='graph1', component_property='figure'),
    Input(component_id='time_index', component_property='value'),
    Input(component_id='var-selector', component_property='value'),
    Input(component_id='profile-selector', component_property='value')
)

def update_graph(time_index, variable, profile_index):
    #if variable == 'SoilMoist_inst': # if user select soil moisture
        #print(variable)
        #selected_var = ds[variable]
        #selected_var = selected_var.isel(SoilMoist_profiles = profile_index)
        #fig = go.Figure(data=[go.Heatmap(z = soilmoist_at_1.isel(time = time_index),  x=longitude, y=latitude)])

    if variable == 'SoilTemp_inst': # if user select soil temperature
       selected_var = ds[variable].sel(SoilTemp_profiles = profile_index)
       print(type(selected_var), selected_var.dims)
       fig = go.Figure(data=[go.Heatmap(z = selected_var.isel(time = time_index),  x=longitude, y=latitude)])

    else: # for any other variables
        selected_var = ds[variable]
        # Plotly graphs
        fig = go.Figure(data=[go.Heatmap(z = selected_var.isel(time = time_index),  x=longitude, y=latitude)])
    
    # Add titles and labels
    fig.update_layout(  
        title=f'{variable}',
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        height = 800,
        width = 1000)
    return fig

In [5]:
# %%
# Run the app
if __name__ == '__main__':
    app.run(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[4], line 18, in update_graph(
    time_index=0,
    variable='SoilTemp_inst',
    profile_index='1'
)
      3 @callback(
      4     Output(component_id='graph1', component_property='figure'),
      5     Input(component_id='time_index', component_property='value'),
   (...)
     14         #selected_var = selected_var.isel(SoilMoist_profiles = profile_index)
     15         #fig = go.Figure(data=[go.Heatmap(z = soilmoist_at_1.isel(time = time_index),  x=longitude, y=latitude)])
     17     if variable == 'SoilTemp_inst': # if user select soil temperature
---> 18        selected_var = ds[variable].sel(SoilTemp_profiles = profile_index)
        variable = 'SoilTemp_inst'
        ds = <xarray.Dataset>
Dimensions:           (time: 4, north_south: 540, east_west: 660,
                       SoilMoist_profiles: 4, SoilTemp_profiles: 

In [1]:
def profile_selection(variable, profile_number):
    if variable in ['SoilMoist_inst', 'SoilTemp_inst']:
        selected_variable = ds[variable]
        if 'SoilMoist_profiles' in selected_variable.dims:
            queried_ds = selected_variable.isel(SoilMoist_profiles = profile_number)
        if 'SoilTemp_profiles' in selected_variable.dims:
            queried_ds = selected_variable.isel(SoilTemp_profiles = profile_number)
        return queried_ds
    else:
        return


In [7]:
profile_selection('SoilMoist_inst', 3)

<xarray.DataArray 'SoilMoist_inst' (time: 4, north_south: 540, east_west: 660)>
[1425600 values with dtype=float32]
Coordinates:
  * time         (time) datetime64[ns] 2024-06-02 2024-06-03 ... 2024-06-05
  * north_south  (north_south) float64 -21.0 -20.95 -20.9 ... 5.9 5.95 6.0
  * east_west    (east_west) float64 -82.0 -81.95 -81.9 ... -49.1 -49.05 -49.0